In [62]:
import pandas as pd
import time
from abc import ABC, abstractmethod

# Custom Exceptions and loggingsetup
class InvalidQuantityError(Exception):
    pass
class InvalidPriceError(Exception):
    pass
import logging
logging.basicConfig(level=logging.INFO)

In [63]:
# Asset Class
class Asset (ABC):
    def __init__(self, name, quantity, purchase_price, current_price):
        if not isinstance (quantity, (int,float)) or quantity <=0:
            raise InvalidQuantityError (f"Quantity must be a positive number, got {quantity}")
        self.validate_price(purchase_price)   # uses static method
        self.validate_price(current_price)
        
        self.name = name
        self.quantity = quantity
        self.purchase_price = purchase_price
        self.current_price = current_price
        logging.info (f"Asset object created. Quantity: {self.quantity}, Name: {self.name}")


    @staticmethod
    def validate_price (price):
        if not isinstance (price, (int,float)) or price <= 0:
            raise InvalidPriceError (f"Price must be positive, got {price}")
        return True
        
    @property
    @abstractmethod
    def current_value(self):
        pass
        
    @property
    def cost_basis(self):
        return self.purchase_price * self.quantity
        
    @property
    def profit_loss(self):
        return self.current_value - self.purchase_price*self.quantity

    def __repr__ (self):
        return f"Name: {self.name}, Quantity: {self.quantity}, Purchase Price: {self.purchase_price}, Current Price: {self.current_price}" 

    def __str__ (self):
        return f"{self.quantity} {self.name} were purchased at {self.purchase_price} and are now worth {self.current_price} each"

In [64]:
# Stock Class
class Stock(Asset):
    def __init__ (self, symbol, shares, purchase_price, current_price):
        super().__init__ (symbol, shares, purchase_price, current_price)

    @property
    def symbol(self):
        return self.name

    @property
    def shares(self):
        return self.quantity

    @property
    def current_value(self):
        return self.current_price * self.shares

    @classmethod
    def from_dict(cls,data):
        sym, shars, purch, curr=data['symbol'],data['shares'], data['purchase_price'], data['current_price']
        return cls(sym, shars, purch, curr)

In [65]:
# Bond Class
class Bond (Asset):
    def __init__ (self, name, quantity, purchase_price, current_price, coupon_rate, maturity_date):
        super().__init__ (name, quantity, purchase_price, current_price)
        self.coupon_rate=coupon_rate
        self.maturity_date=maturity_date

    @property
    def current_value(self):
        return self.current_price * self.quantity

In [66]:
# Real Estate
class RealEstate(Asset):
    def __init__ (self, name, quantity, purchase_price, current_price, location, square_feet):
        super().__init__ (name, quantity, purchase_price, current_price)
        self.location=location
        self.square_feet=square_feet
    @property
    def current_value(self):
        return self.current_price * self.quantity

In [67]:
# Timing decorator
def time_decorator(func):
        def wrapper (*args, **kwargs):
            start=time.time()
            result=func(*args,**kwargs)
            end=time.time()
            logging.info(f"{func.__name__}executed in {end-start:.4f} seconds")
            return result
        return wrapper

In [70]:
#Portfolio Class
class Portfolio:
    def __init__(self,name):
        self.name=name
        self.holdings=[]
        logging.info (f"{self.name} Portfolio object created.")
    def add_stock(self,stock):
        self.holdings.append(stock)
    def remove_stock (self,symbol):
        initial_count=len(self.holdings)
        self.holdings=list(filter(lambda stock: stock.symbol!=symbol,self.holdings))
        end_count= len(self.holdings)
        if initial_count-end_count==0:
            raise ValueError(f"No Stock of symbol {symbol} exists in the Portfolio")
        logging.info (f"{symbol} Stock removed from {self.name} Portfolio")
    @time_decorator
    def total_value(self):
        return sum(stock.current_value for stock in self.holdings)
    def total_cost_basis(self):
        return sum(stock.cost_basis for stock in self.holdings)
    def total_profit_loss(self):
        return self.total_value()-self.total_cost_basis() 
    def get_holdings_summary(self):
        summary=[]
        for stock in self.holdings:
            summary.append({
                "symbol": stock.symbol,
                "shares": stock.shares,
                "current_value": stock.current_value,
                "profit_loss": stock.profit_loss
            })
        return summary

    def __add__(self, other):
        new_portfolio= Portfolio("MegredPortfolio")
        new_portfolio.holdings=self.holdings + other.holdings
        return new_portfolio
    def __iadd__ (self,other):
        self.holdings.extend(other.holdings)
        return self
        
    def __repr__(self):
        return f"Name: {self.name}, Holdings: {len(self.holdings)}"
    def __len__(self):
        return len(self.holdings)
    def __getitem__(self,index):
        return self.holdings[index]

    def load_from_csv(self, filename):
        try:
            df = pd.read_csv(filename)
        except FileNotFoundError:
            logging.error(f"File not found: {filename}")
            raise
        for _, row in df.iterrows():
            stock_data={
                "symbol":row["symbol"],
                'shares': row['shares'],
                'purchase_price': row['purchase_price'],
                'current_price': row['current_price']
        }
            stock=Stock.from_dict(stock_data)
            self.add_stock(stock)
        logging.info(f"Loaded {len(df)} stocks from {filename}")

    def filter_stocks(self, min_value=0):
        filtered_stocks=list(filter(lambda s: s.current_value > min_value, self.holdings))
        symbols=list(map(lambda y: y.symbol, filtered_stocks))
        return filtered_stocks, symbols

    def display_holdings(self):
        for index,holding in enumerate(self.holdings, start=1):
            print(f"{index}. {holding.symbol}")

In [69]:
if __name__ == "__main__":
    # Create a few assets
    aapl = Stock("AAPL", 10, 150, 175)
    msft = Stock("MSFT", 5, 200, 210)
    bond = Bond("US10Y", 1000, 95, 98, 0.02, "2030-12-31")
    realty = RealEstate("Warehouse", 1, 500000, 550000, "Chicago", 10000)

    # Build a portfolio
    p = Portfolio("My Portfolio")
    p.add_stock(aapl)
    p.add_stock(msft)
    p.add_stock(bond)
    p.add_stock(realty)

    # Quick checks
    print("Holdings:", [asset.name for asset in p.holdings])
    print("Total value:", p.total_value())
    print("Total cost:", p.total_cost_basis())
    print("Profit/loss:", p.total_profit_loss())

    # Test merge
    p2 = Portfolio("Another")
    p2.add_stock(Stock("TSLA", 2, 600, 650))
    p3 = p + p2
    print("Merged size:", len(p3))

INFO:root:Asset object created. Quantity: 10, Name: AAPL
INFO:root:Asset object created. Quantity: 5, Name: MSFT
INFO:root:Asset object created. Quantity: 1000, Name: US10Y
INFO:root:Asset object created. Quantity: 1, Name: Warehouse
INFO:root:My Portfolio Portfolio object created.
INFO:root:total_valueexecuted in 0.0000 seconds
INFO:root:total_valueexecuted in 0.0000 seconds
INFO:root:Another Portfolio object created.
INFO:root:Asset object created. Quantity: 2, Name: TSLA
INFO:root:MegredPortfolio Portfolio object created.


Holdings: ['AAPL', 'MSFT', 'US10Y', 'Warehouse']
Total value: 650800
Total cost: 597500
Profit/loss: 53300
Merged size: 5
